In [4]:
import torch
import pandas as pd
import sqlite3
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import softmax
from rdkit import Chem
from Bio import SeqIO
import random
import re
from transformers import T5EncoderModel, T5Tokenizer
import gzip  # <-- FIX 1: ADDED THIS IMPORT

# --- 1. System & Configuration ---

# --- Configuration (UPDATE THESE PATHS & ID) ---
CHEMPL_DB_PATH = 'DL_ENDSEM__DATASET/chembl_35/chembl_35_sqlite/chembl_35.db'
BLAST_FASTA_PATH = 'DL_ENDSEM__DATASET/chembl_35_blast.fa.gz'
TARGET_UNIPROT_ID = "P00533" # Example: EGFR Kinase

# --- Model Hyperparameters ---
Z_DIM = 100         # Latent noise dimension
ATOM_FEAT_DIM = 9   # Atom feature size
BOND_FEAT_DIM = 4   # Bond feature size (Single, Double, Triple, Aromatic)
EMBED_DIM = 128     # Hidden dimension for the Graph Transformer
T_EMBED_DIM = 1024  # Target embedding dimension (from ProtT5)
LAMBDA_GP = 10.0    # Gradient Penalty weight
MAX_NODES = 30      # Max atoms in generated molecules (for Generator tensor shape)
N_CRITIC = 5        # Discriminator training steps per Generator step
EPOCHS = 10       # Total epochs
BATCH_SIZE = 128

# --- CUDA Check ---
if torch.cuda.is_available():
    print("✅ CUDA is available! GPU will be used for training.")
    print(f"PyTorch CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    DEVICE = torch.device('cuda')
else:
    print("❌ CUDA not found. Running on CPU.")
    DEVICE = torch.device('cpu')
print(f"Using device: {DEVICE}")


# --- 2. Real Protein Embedding Generation ---

# --- FIX 2: REPLACED THIS FUNCTION ---
def load_target_sequence(fasta_path, uniprot_id):
    """Loads a protein sequence from a gzipped FASTA file."""
    print(f"Opening gzipped FASTA file: {fasta_path}")
    try:
        # 'rt' = read in text mode
        with gzip.open(fasta_path, "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                if uniprot_id in record.id or uniprot_id in record.description:
                    return str(record.seq)
        print(f"Warning: Could not find sequence for {uniprot_id} in {fasta_path}")
        return None
    except FileNotFoundError:
        print(f"FATAL ERROR: FASTA file not found at {fasta_path}")
        raise
    except Exception as e:
        print(f"FATAL ERROR: Could not read FASTA file. Error: {e}")
        raise

def get_protein_embedding(sequence, device):
    """
    Generates a protein embedding using the pre-trained ProtT5 model.
    This replaces the "mock" random embedding.
    """
    print("Loading ProtT5 model... (This may take a moment)")
    # Load model and tokenizer
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)
    model = T5EncoderModel.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc').to(device)
    model.eval() # Set to evaluation mode

    # Pre-process sequence: add spaces between AAs and handle rare AAs
    sequence_preprocessed = " ".join(list(re.sub(r"[UZOB]", "X", sequence)))
    
    inputs = tokenizer(sequence_preprocessed, return_tensors="pt", max_length=1024, truncation=True).to(device)
    
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state
    
    # Average pool the embedding to get a single vector [1, 1024]
    protein_vec = embedding.mean(dim=1).squeeze(0) # Squeeze to [1024]
    print(f"Generated protein embedding of shape: {protein_vec.shape}")
    return protein_vec

# --- Generate the REAL Target Embedding ---
target_seq = load_target_sequence(BLAST_FASTA_PATH, TARGET_UNIPROT_ID)
if target_seq is None:
    raise ValueError(f"Target sequence for {TARGET_UNIPROT_ID} not found. Exiting.")

TARGET_EMBED = get_protein_embedding(target_seq, DEVICE)


# --- 3. Data Pipeline (Molecules -> Graphs) ---

def extract_potent_inhibitors(db_path, uniprot_id, potency_cutoff_nM=100):
    """
    Connects to ChEMBL DB and extracts SMILES for potent inhibitors of a given Uniprot ID.
    (Using your robust SQL query)
    """
    try:
        conn = sqlite3.connect(db_path)
        sql_query = f"""
        SELECT DISTINCT
            cs.canonical_smiles
        FROM
            activities acts
        JOIN assays a ON acts.assay_id = a.assay_id
        JOIN target_dictionary td ON a.tid = td.tid
        JOIN target_components tc ON td.tid = tc.tid
        JOIN component_sequences cseq ON tc.component_id = cseq.component_id
        JOIN compound_structures cs ON acts.molregno = cs.molregno
        WHERE
            cseq.accession = '{uniprot_id}' AND
            acts.standard_type = 'IC50' AND
            acts.standard_units = 'nM' AND
            acts.standard_relation = '=' AND
            acts.standard_value <= {potency_cutoff_nM}
        """
        df = pd.read_sql_query(sql_query, conn)
        conn.close()
        
        print(f"Found {len(df)} potent inhibitors for UniProt ID {uniprot_id}.")
        return df['canonical_smiles'].unique().tolist()
        
    except Exception as e:
        print(f"Error during database query. This is likely due to missing tables or a critical file path issue.")
        raise RuntimeError(f"Database Error: {e}. Please ensure the file is the full ChEMBL SQLite dump.") from e

def get_atom_features(atom):
    """Creates the 9-dimensional atom feature vector."""
    return [
        atom.GetAtomicNum(), atom.GetDegree(), atom.GetFormalCharge(),
        int(atom.GetIsAromatic()),
        int(atom.GetHybridization() == Chem.HybridizationType.SP),
        int(atom.GetHybridization() == Chem.HybridizationType.SP2),
        int(atom.GetHybridization() == Chem.HybridizationType.SP3),
        int(atom.GetHybridization() == Chem.HybridizationType.SP3D),
        int(atom.GetHybridization() == Chem.HybridizationType.SP3D2)
    ]

def smiles_to_graph(smiles, target_embed):
    """Converts SMILES to a PyG Data object."""
    mol = Chem.MolFromSmiles(smiles)
    if not mol: return None
    if mol.GetNumAtoms() > MAX_NODES: return None

    x = torch.tensor([get_atom_features(atom) for atom in mol.GetAtoms()], dtype=torch.float).to(DEVICE)
    
    edge_indices, edge_attrs = [], []
    bond_types = [Chem.BondType.SINGLE, Chem.BondType.DOUBLE, Chem.BondType.TRIPLE, Chem.BondType.AROMATIC]
    
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        bond_type_oh = [int(bond.GetBondType() == t) for t in bond_types]
        edge_indices.extend([[i, j], [j, i]])
        edge_attrs.extend([bond_type_oh, bond_type_oh])

    if not edge_indices: return None
    
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous().to(DEVICE)
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float).to(DEVICE)

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, 
                target_embed=target_embed.unsqueeze(0))
    return data

# --- Data Pipeline Execution ---
inhibitor_smiles = extract_potent_inhibitors(CHEMPL_DB_PATH, TARGET_UNIPROT_ID)
real_data_list = [smiles_to_graph(s, TARGET_EMBED) for s in inhibitor_smiles]
real_data_list = [d for d in real_data_list if d is not None]

if not real_data_list:
    print("FATAL: No valid inhibitor data found. The model cannot be trained.")
    exit()

real_loader = DataLoader(real_data_list, batch_size=BATCH_SIZE, shuffle=True)
print(f"Prepared {len(real_data_list)} real graph samples for training.")


# --- 4. Model Architecture ---

# --- 4.1. Relational Graph Transformer Layer (No Changes) ---
class RelationalGraphTransformerLayer(MessagePassing):
    """Graph Transformer Layer with explicit edge/bond feature integration."""
    def __init__(self, in_channels, out_channels, edge_dim, heads=4, dropout=0.1, **kwargs):
        super().__init__(aggr='add', node_dim=0, **kwargs)
        self.out_channels = out_channels
        self.heads = heads
        
        self.lin_q = nn.Linear(in_channels, heads * out_channels, bias=False)
        self.lin_k = nn.Linear(in_channels, heads * out_channels, bias=False)
        self.lin_v = nn.Linear(in_channels, heads * out_channels, bias=False)
        self.lin_edge = nn.Linear(edge_dim, heads * out_channels, bias=False) 
        self.att_coeff = nn.Parameter(torch.Tensor(1, heads, 2 * out_channels)) 
        self.lin_out = nn.Linear(heads * out_channels, out_channels)
        self.dropout = nn.Dropout(dropout)
        
        # Initialize attention coefficients (Xavier initialization)
        nn.init.xavier_uniform_(self.att_coeff)

    def forward(self, x, edge_index, edge_attr):
        Q = self.lin_q(x).view(-1, self.heads, self.out_channels)
        K = self.lin_k(x).view(-1, self.heads, self.out_channels)
        V = self.lin_v(x).view(-1, self.heads, self.out_channels)
        E = self.lin_edge(edge_attr).view(-1, self.heads, self.out_channels)

        out = self.propagate(edge_index, Q=Q, K=K, V=V, E=E)
        out = out.view(-1, self.heads * self.out_channels)
        out = self.lin_out(out)
        return out

    def message(self, Q_i, K_j, V_j, E_k, index):
        try:
            is_empty = (E_k.size(0) == 0)
        except AttributeError:
            is_empty = True
        
        if is_empty:
            E_bias = torch.zeros(Q_i.size(0), self.heads, 1, device=self.lin_q.weight.device)
        else:
            E_bias = E_k.mean(dim=-1, keepdim=True) 

        QK_cat = torch.cat([Q_i, K_j], dim=-1)
        e_ij = (QK_cat * self.att_coeff).sum(dim=-1, keepdim=True)
        e_ij = e_ij + E_bias
        
        e_ij = F.leaky_relu(e_ij)
        alpha = softmax(e_ij, index)
        alpha = self.dropout(alpha)
        
        return V_j * alpha.view(-1, self.heads, 1)

# --- 4.2. Discriminator (No Changes) ---
class Discriminator(nn.Module):
    def __init__(self, node_features, edge_dim, t_embed_dim, embed_dim, num_layers=3):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            in_d = node_features if i == 0 else embed_dim
            self.layers.append(RelationalGraphTransformerLayer(in_d, embed_dim, edge_dim))
        self.lin_final = nn.Linear(embed_dim + t_embed_dim, 1)
        
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        t_embed = data.target_embed
        
        for layer in self.layers:
            x = layer(x, edge_index, edge_attr)
            x = F.relu(x)
        
        graph_embed = global_mean_pool(x, batch)
        
        # Ensure t_embed is [batch_size, t_embed_dim]
        if t_embed.dim() > 2:
            t_embed = t_embed.squeeze(1) 
            
        final_input = torch.cat([graph_embed, t_embed], dim=1)
        
        return self.lin_final(final_input).squeeze(1)

# --- 4.3. Generator (FIXED: Outputs Logits) ---
class Generator(nn.Module):
    def __init__(self, z_dim, t_embed_dim, node_features, bond_features, max_nodes=MAX_NODES):
        super().__init__()
        self.max_nodes = max_nodes
        self.node_features = node_features
        self.bond_features = bond_features
        
        self.lin_x = nn.Sequential(
            nn.Linear(z_dim + t_embed_dim, 256), nn.ReLU(),
            nn.Linear(256, max_nodes * node_features)
        )
        self.lin_adj = nn.Sequential(
            nn.Linear(z_dim + t_embed_dim, 256), nn.ReLU(),
            nn.Linear(256, max_nodes * max_nodes * bond_features) 
        )

    def forward(self, z, t_embed):
        zt = torch.cat([z, t_embed], dim=1)
        
        x_fake = self.lin_x(zt).view(-1, self.max_nodes, self.node_features)
        
        # Output raw logits for the adjacency matrix (NO SOFTMAX)
        adj_fake_logits = self.lin_adj(zt).view(-1, self.max_nodes, self.max_nodes, self.bond_features)
        
        # Note: We don't softmax x_fake as it contains continuous features (e.g., atomic num)
        
        return x_fake, adj_fake_logits

# --- Model Initialization ---
print("Initializing models...")
generator = Generator(Z_DIM, T_EMBED_DIM, ATOM_FEAT_DIM, BOND_FEAT_DIM).to(DEVICE)
discriminator = Discriminator(ATOM_FEAT_DIM, BOND_FEAT_DIM, T_EMBED_DIM, EMBED_DIM).to(DEVICE)

optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.9))
optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.9))


# --- 5. Training Utilities (FIXED) ---

# --- 5.1. Differentiable Graph Conversion (NEW) ---
def convert_fake_to_data_differentiable(x_fake_tensor, adj_fake_logits, t_embed_batch, device, temperature=0.5):
    """
    Converts Generator output to PyG Data using Gumbel-Softmax for the G-step.
    This IS differentiable, allowing the generator to learn bond formation.
    """
    batch_size = x_fake_tensor.size(0)
    data_list = []
    
    for i in range(batch_size):
        num_nodes = MAX_NODES
        x_i = x_fake_tensor[i, :, :]
        adj_i_logits = adj_fake_logits[i, :, :, :]
        
        edge_indices, edge_attrs_gumbel = [], []
        
        for r in range(num_nodes):
            for c in range(num_nodes):
                if r == c: continue
                
                # Gumbel-Softmax Sampling (Differentiable "argmax")
                bond_probs = F.gumbel_softmax(
                    adj_i_logits[r, c], 
                    tau=temperature, 
                    hard=True
                )
                
                edge_indices.append([r, c])
                edge_attrs_gumbel.append(bond_probs)

        if not edge_indices: continue

        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous().to(device)
        edge_attr = torch.stack(edge_attrs_gumbel).to(device)

        data_list.append(Data(x=x_i, edge_index=edge_index, edge_attr=edge_attr, 
                              target_embed=t_embed_batch[i].unsqueeze(0)))
    
    if not data_list: return None
    # We return the .dataset attribute for direct use in GP calculation
    return DataLoader(data_list, batch_size=batch_size).dataset

# --- 5.2. Discrete Graph Conversion (NEW) ---
def convert_fake_to_data_discrete(x_fake_tensor, adj_fake_logits, t_embed_batch, device):
    """
    Converts Generator output to PyG Data using .argmax() for the D-step.
    This is NOT differentiable and is used when we don't need grads (faster).
    """
    batch_size = x_fake_tensor.size(0)
    data_list = []
    
    for i in range(batch_size):
        num_nodes = MAX_NODES
        x_i = x_fake_tensor[i, :, :].detach() # Detach all inputs
        adj_i_logits = adj_fake_logits[i, :, :, :].detach()
        
        edge_indices, edge_attrs_list = [], []
        
        for r in range(num_nodes):
            for c in range(num_nodes):
                if r == c: continue
                
                # Hard .argmax() sampling
                bond_type_index = adj_i_logits[r, c].argmax().item()
                bond_one_hot = F.one_hot(
                    torch.tensor(bond_type_index, device=device), 
                    num_classes=BOND_FEAT_DIM
                ).float()
                
                edge_indices.append([r, c])
                edge_attrs_list.append(bond_one_hot)

        if not edge_indices: continue
        
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous().to(device)
        edge_attr = torch.stack(edge_attrs_list).to(device)

        data_list.append(Data(x=x_i, edge_index=edge_index, edge_attr=edge_attr, 
                              target_embed=t_embed_batch[i].unsqueeze(0)))
    
    if not data_list: return None
    return DataLoader(data_list, batch_size=batch_size).dataset

# --- 5.3. WGAN-GP Gradient Penalty (Preserved User's Fix) ---
def calculate_gradient_penalty(discriminator, real_data, fake_data, lambda_gp, device):
    """Calculates the Gradient Penalty on interpolated node features (X)."""
    
    real_x = real_data.x.detach()
    fake_x = fake_data.x.detach()
    real_x_size = real_x.size(0)
    
    # --- FIX: Match fake_data.x size to real_data.x size for interpolation ---
    if fake_x.size(0) > real_x_size:
        fake_x = fake_x[:real_x_size]
    elif fake_x.size(0) < real_x_size:
        padding = torch.zeros(real_x_size - fake_x.size(0), fake_x.size(1), device=device)
        fake_x = torch.cat([fake_x, padding], dim=0)

    # 1. Linear Interpolation
    alpha = torch.rand(real_x_size, 1).to(device) 
    interpolated_x = (alpha * real_x) + ((1 - alpha) * fake_x)
    interpolated_x.requires_grad_(True)

    # 2. Create interpolated Data object
    # We use the real data's structure (edges, batch) as a template
    interpolated_data = Data(x=interpolated_x, 
                             edge_index=real_data.edge_index, 
                             edge_attr=real_data.edge_attr, 
                             batch=real_data.batch, 
                             target_embed=real_data.target_embed)

    disc_interpolates = discriminator(interpolated_data)
    
    gradients = torch.autograd.grad(
        outputs=disc_interpolates, inputs=interpolated_x,
        grad_outputs=torch.ones_like(disc_interpolates),
        create_graph=True, retain_graph=True
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_norm = gradients.norm(2, dim=1)
    gradient_penalty = lambda_gp * ((gradient_norm - 1) ** 2).mean()
    
    return gradient_penalty

# --- 6. Main Training Loop (FIXED) ---
def run_wgan_gp_training(generator, discriminator, data_loader, epochs, n_critic):
    
    generator.train()
    discriminator.train()
    
    for epoch in range(1, epochs + 1):
        g_loss_sum, d_loss_sum = 0, 0
        
        for batch_idx, real_data in enumerate(data_loader):
            # --- THIS IS THE LINE YOU REQUESTED ---
            print(f"--- Epoch {epoch}, Processing Batch {batch_idx+1}/{len(data_loader)} ---")

            real_data = real_data.to(DEVICE)
            batch_size = real_data.num_graphs
            
            # Ensure target_embed_batch is [batch_size, T_EMBED_DIM]
            target_embed_batch = real_data.target_embed
            if target_embed_batch.dim() > 2:
                target_embed_batch = target_embed_batch.view(batch_size, T_EMBED_DIM)

            # 1. Train Discriminator (n_critic steps)
            for _ in range(n_critic):
                optimizer_D.zero_grad()
                
                # Real Loss
                d_real = discriminator(real_data).mean()
                
                # Fake Loss
                z = torch.randn(batch_size, Z_DIM).to(DEVICE)
                x_fake, adj_fake_logits = generator(z, target_embed_batch)
                
                # --- D-step uses DISCRETE (non-differentiable) sampling ---
                fake_data_list = convert_fake_to_data_discrete(
                    x_fake.detach(), adj_fake_logits.detach(), target_embed_batch, DEVICE
                )
                if fake_data_list is None: continue
                # Convert list of Data objects back into a batched Data object
                fake_data_loader = DataLoader(fake_data_list, batch_size=batch_size)
                fake_data = next(iter(fake_data_loader)).to(DEVICE)

                d_fake = discriminator(fake_data).mean()
                
                # Gradient Penalty
                gp = calculate_gradient_penalty(discriminator, real_data, fake_data, LAMBDA_GP, DEVICE)
                
                d_loss = - (d_real - d_fake) + gp
                d_loss.backward()
                optimizer_D.step()
                d_loss_sum += d_loss.item()
            
            # 2. Train Generator (1 step)
            optimizer_G.zero_grad()
            
            z = torch.randn(batch_size, Z_DIM).to(DEVICE)
            x_fake, adj_fake_logits = generator(z, target_embed_batch)
            
            # --- G-step uses DIFFERENTIABLE Gumbel-Softmax sampling ---
            fake_data_list = convert_fake_to_data_differentiable(
                x_fake, adj_fake_logits, target_embed_batch, DEVICE
            )
            if fake_data_list is None: continue
            fake_data_loader = DataLoader(fake_data_list, batch_size=batch_size)
            fake_data = next(iter(fake_data_loader)).to(DEVICE)
            
            # Generator Loss
            g_loss = - discriminator(fake_data).mean()
            g_loss.backward()
            optimizer_G.step()
            g_loss_sum += g_loss.item()
            
        avg_d_loss = d_loss_sum / len(data_loader) / n_critic
        avg_g_loss = g_loss_sum / len(data_loader)
        print(f"Epoch {epoch}/{EPOCHS} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")

# --- Execute Training ---
print("\n--- Starting WGAN-GP Training ---")
run_wgan_gp_training(generator, discriminator, real_loader, EPOCHS, N_CRITIC) 
print("\nTraining completed.")

✅ CUDA is available! GPU will be used for training.
PyTorch CUDA Version: 12.1
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU
Using device: cuda
Opening gzipped FASTA file: DL_ENDSEM__DATASET/chembl_35_blast.fa.gz
Loading ProtT5 model... (This may take a moment)
Generated protein embedding of shape: torch.Size([1024])
Found 3989 potent inhibitors for UniProt ID P00533.
Prepared 1334 real graph samples for training.
Initializing models...

--- Starting WGAN-GP Training ---
--- Epoch 1, Processing Batch 1/11 ---
--- Epoch 1, Processing Batch 2/11 ---
--- Epoch 1, Processing Batch 3/11 ---
--- Epoch 1, Processing Batch 4/11 ---
--- Epoch 1, Processing Batch 5/11 ---
--- Epoch 1, Processing Batch 6/11 ---
--- Epoch 1, Processing Batch 7/11 ---
--- Epoch 1, Processing Batch 8/11 ---
--- Epoch 1, Processing Batch 9/11 ---
--- Epoch 1, Processing Batch 10/11 ---
--- Epoch 1, Processing Batch 11/11 ---
Epoch 1/10 | D Loss: -12.3766 | G Loss: -0.0959
--- Epoch 2, Processing Batch 1/11 ---
--- Epo

KeyboardInterrupt: 

In [1]:
import torch
import pandas as pd
import sqlite3
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import softmax
from rdkit import Chem
from Bio import SeqIO
import random
import re
from transformers import T5EncoderModel, T5Tokenizer
import gzip

# --- 1. System & Configuration ---

# --- Configuration (UPDATE THESE PATHS & ID) ---
CHEMPL_DB_PATH = 'DL_ENDSEM__DATASET/chembl_35/chembl_35_sqlite/chembl_35.db'
BLAST_FASTA_PATH = 'DL_ENDSEM__DATASET/chembl_35_blast.fa.gz'
TARGET_UNIPROT_ID = "P00533" # Example: EGFR Kinase

# --- Model Hyperparameters ---
Z_DIM = 100         # Latent noise dimension
ATOM_FEAT_DIM = 9   # Atom feature size
BOND_FEAT_DIM = 4   # Bond feature size (Single, Double, Triple, Aromatic)
EMBED_DIM = 128     # Hidden dimension for the Graph Transformer
T_EMBED_DIM = 1024  # Target embedding dimension (from ProtT5)
LAMBDA_GP = 10.0    # Gradient Penalty weight
MAX_NODES = 30      # Max atoms in generated molecules (for Generator tensor shape)
N_CRITIC = 5        # Discriminator training steps per Generator step
EPOCHS = 10         # Total epochs
BATCH_SIZE = 64    # Your increased batch size

# --- OPTIMIZATION 1: Set num_workers based on your 16-core CPU ---
# Use ~half your cores to pre-fetch real data
CPU_WORKERS = 4  

# --- CUDA Check ---
if torch.cuda.is_available():
    print("✅ CUDA is available! GPU will be used for training.")
    print(f"PyTorch CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    DEVICE = torch.device('cuda')
else:
    print("❌ CUDA not found. Running on CPU.")
    DEVICE = torch.device('cpu')
print(f"Using device: {DEVICE}")


# --- 2. Real Protein Embedding Generation (No changes) ---

def load_target_sequence(fasta_path, uniprot_id):
    """Loads a protein sequence from a gzipped FASTA file."""
    print(f"Opening gzipped FASTA file: {fasta_path}")
    try:
        with gzip.open(fasta_path, "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                if uniprot_id in record.id or uniprot_id in record.description:
                    return str(record.seq)
        print(f"Warning: Could not find sequence for {uniprot_id} in {fasta_path}")
        return None
    except FileNotFoundError:
        print(f"FATAL ERROR: FASTA file not found at {fasta_path}")
        raise
    except Exception as e:
        print(f"FATAL ERROR: Could not read FASTA file. Error: {e}")
        raise

def get_protein_embedding(sequence, device):
    """Generates a protein embedding using the pre-trained ProtT5 model."""
    print("Loading ProtT5 model... (This may take a moment)")
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)
    model = T5EncoderModel.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc').to(device)
    model.eval() 

    sequence_preprocessed = " ".join(list(re.sub(r"[UZOB]", "X", sequence)))
    inputs = tokenizer(sequence_preprocessed, return_tensors="pt", max_length=1024, truncation=True).to(device)
    
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state
    
    protein_vec = embedding.mean(dim=1).squeeze(0)
    print(f"Generated protein embedding of shape: {protein_vec.shape}")
    return protein_vec

# --- Generate the REAL Target Embedding ---
target_seq = load_target_sequence(BLAST_FASTA_PATH, TARGET_UNIPROT_ID)
if target_seq is None:
    raise ValueError(f"Target sequence for {TARGET_UNIPROT_ID} not found. Exiting.")
TARGET_EMBED = get_protein_embedding(target_seq, DEVICE)


# --- 3. Data Pipeline (Molecules -> Graphs) (No changes) ---

def extract_potent_inhibitors(db_path, uniprot_id, potency_cutoff_nM=100):
    """Connects to ChEMBL DB and extracts SMILES for potent inhibitors."""
    try:
        conn = sqlite3.connect(db_path)
        sql_query = f"""
        SELECT DISTINCT cs.canonical_smiles
        FROM activities acts
        JOIN assays a ON acts.assay_id = a.assay_id
        JOIN target_dictionary td ON a.tid = td.tid
        JOIN target_components tc ON td.tid = tc.tid
        JOIN component_sequences cseq ON tc.component_id = cseq.component_id
        JOIN compound_structures cs ON acts.molregno = cs.molregno
        WHERE
            cseq.accession = '{uniprot_id}' AND
            acts.standard_type = 'IC50' AND
            acts.standard_units = 'nM' AND
            acts.standard_relation = '=' AND
            acts.standard_value <= {potency_cutoff_nM}
        """
        df = pd.read_sql_query(sql_query, conn)
        conn.close()
        print(f"Found {len(df)} potent inhibitors for UniProt ID {uniprot_id}.")
        return df['canonical_smiles'].unique().tolist()
    except Exception as e:
        print(f"Error during database query: {e}")
        raise

def get_atom_features(atom):
    """Creates the 9-dimensional atom feature vector."""
    return [
        atom.GetAtomicNum(), atom.GetDegree(), atom.GetFormalCharge(),
        int(atom.GetIsAromatic()),
        int(atom.GetHybridization() == Chem.HybridizationType.SP),
        int(atom.GetHybridization() == Chem.HybridizationType.SP2),
        int(atom.GetHybridization() == Chem.HybridizationType.SP3),
        int(atom.GetHybridization() == Chem.HybridizationType.SP3D),
        int(atom.GetHybridization() == Chem.HybridizationType.SP3D2)
    ]

def smiles_to_graph(smiles, target_embed):
    """Converts SMILES to a PyG Data object."""
    mol = Chem.MolFromSmiles(smiles)
    if not mol: return None
    if mol.GetNumAtoms() > MAX_NODES: return None

    x = torch.tensor([get_atom_features(atom) for atom in mol.GetAtoms()], dtype=torch.float)
    
    edge_indices, edge_attrs = [], []
    bond_types = [Chem.BondType.SINGLE, Chem.BondType.DOUBLE, Chem.BondType.TRIPLE, Chem.BondType.AROMATIC]
    
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        bond_type_oh = [int(bond.GetBondType() == t) for t in bond_types]
        edge_indices.extend([[i, j], [j, i]])
        edge_attrs.extend([bond_type_oh, bond_type_oh])

    if not edge_indices: return None
    
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

    # Note: We send to DEVICE in the data loader, not here.
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, 
                target_embed=target_embed.unsqueeze(0))
    return data

# --- Data Pipeline Execution ---
inhibitor_smiles = extract_potent_inhibitors(CHEMPL_DB_PATH, TARGET_UNIPROT_ID)
# Note: We keep data on CPU first, for num_workers to be efficient.
real_data_list = [smiles_to_graph(s, TARGET_EMBED.cpu()) for s in inhibitor_smiles]
real_data_list = [d for d in real_data_list if d is not None]

if not real_data_list:
    print("FATAL: No valid inhibitor data found. The model cannot be trained.")
    exit()

# --- OPTIMIZATION 2: Added num_workers and pin_memory ---
real_loader = DataLoader(
    real_data_list, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=CPU_WORKERS, 
    pin_memory=True  # Speeds up CPU-to-GPU data transfer
)
print(f"Prepared {len(real_data_list)} real graph samples for training.")


# --- 4. Model Architecture (No changes) ---

# --- 4.1. Relational Graph Transformer Layer ---
class RelationalGraphTransformerLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, edge_dim, heads=4, dropout=0.1, **kwargs):
        super().__init__(aggr='add', node_dim=0, **kwargs)
        self.out_channels = out_channels
        self.heads = heads
        self.lin_q = nn.Linear(in_channels, heads * out_channels, bias=False)
        self.lin_k = nn.Linear(in_channels, heads * out_channels, bias=False)
        self.lin_v = nn.Linear(in_channels, heads * out_channels, bias=False)
        self.lin_edge = nn.Linear(edge_dim, heads * out_channels, bias=False) 
        self.att_coeff = nn.Parameter(torch.Tensor(1, heads, 2 * out_channels)) 
        self.lin_out = nn.Linear(heads * out_channels, out_channels)
        self.dropout = nn.Dropout(dropout)
        nn.init.xavier_uniform_(self.att_coeff)

    def forward(self, x, edge_index, edge_attr):
        Q = self.lin_q(x).view(-1, self.heads, self.out_channels)
        K = self.lin_k(x).view(-1, self.heads, self.out_channels)
        V = self.lin_v(x).view(-1, self.heads, self.out_channels)
        E = self.lin_edge(edge_attr).view(-1, self.heads, self.out_channels)
        out = self.propagate(edge_index, Q=Q, K=K, V=V, E=E)
        out = out.view(-1, self.heads * self.out_channels)
        out = self.lin_out(out)
        return out

    def message(self, Q_i, K_j, V_j, E_k, index):
        try: is_empty = (E_k.size(0) == 0)
        except AttributeError: is_empty = True
        
        if is_empty: E_bias = torch.zeros(Q_i.size(0), self.heads, 1, device=self.lin_q.weight.device)
        else: E_bias = E_k.mean(dim=-1, keepdim=True) 

        QK_cat = torch.cat([Q_i, K_j], dim=-1)
        e_ij = (QK_cat * self.att_coeff).sum(dim=-1, keepdim=True)
        e_ij = e_ij + E_bias
        e_ij = F.leaky_relu(e_ij)
        alpha = softmax(e_ij, index)
        alpha = self.dropout(alpha)
        return V_j * alpha.view(-1, self.heads, 1)

# --- 4.2. Discriminator ---
class Discriminator(nn.Module):
    def __init__(self, node_features, edge_dim, t_embed_dim, embed_dim, num_layers=3):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            in_d = node_features if i == 0 else embed_dim
            self.layers.append(RelationalGraphTransformerLayer(in_d, embed_dim, edge_dim))
        self.lin_final = nn.Linear(embed_dim + t_embed_dim, 1)
        
    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        t_embed = data.target_embed
        
        for layer in self.layers:
            x = layer(x, edge_index, edge_attr)
            x = F.relu(x)
        
        graph_embed = global_mean_pool(x, batch)
        if t_embed.dim() > 2: t_embed = t_embed.squeeze(1) 
        final_input = torch.cat([graph_embed, t_embed], dim=1)
        return self.lin_final(final_input).squeeze(1)

# --- 4.3. Generator ---
class Generator(nn.Module):
    def __init__(self, z_dim, t_embed_dim, node_features, bond_features, max_nodes=MAX_NODES):
        super().__init__()
        self.max_nodes = max_nodes
        self.node_features = node_features
        self.bond_features = bond_features
        self.lin_x = nn.Sequential(nn.Linear(z_dim + t_embed_dim, 256), nn.ReLU(), nn.Linear(256, max_nodes * node_features))
        self.lin_adj = nn.Sequential(nn.Linear(z_dim + t_embed_dim, 256), nn.ReLU(), nn.Linear(256, max_nodes * max_nodes * bond_features))

    def forward(self, z, t_embed):
        zt = torch.cat([z, t_embed], dim=1)
        x_fake = self.lin_x(zt).view(-1, self.max_nodes, self.node_features)
        adj_fake_logits = self.lin_adj(zt).view(-1, self.max_nodes, self.max_nodes, self.bond_features)
        return x_fake, adj_fake_logits

# --- Model Initialization ---
print("Initializing models...")
generator = Generator(Z_DIM, T_EMBED_DIM, ATOM_FEAT_DIM, BOND_FEAT_DIM).to(DEVICE)
discriminator = Discriminator(ATOM_FEAT_DIM, BOND_FEAT_DIM, T_EMBED_DIM, EMBED_DIM).to(DEVICE)

optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.9))
optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.9))


# --- 5. Training Utilities (FIXED & VECTORIZED) ---

# --- OPTIMIZATION 3: Create a helper for the dense edge_index template ---
# We compute this once on CPU and move it to GPU,
# so it's not recomputed every batch.
N = MAX_NODES
# Create a [N, N] adjacency matrix with 1s everywhere except the diagonal
adj_template = (torch.ones(N, N) - torch.eye(N)).bool()
# Convert to sparse edge_index format [2, N*(N-1)]
EDGE_INDEX_TEMPLATE = adj_template.nonzero(as_tuple=False).t().contiguous().to(DEVICE)
# This template has N*(N-1) edges
NUM_DENSE_EDGES = EDGE_INDEX_TEMPLATE.size(1)

# --- OPTIMIZATION 4: Vectorized Fake Graph Generation ---
# This one function replaces both old convert_fake_to_data functions
# It's fully vectorized and runs on the GPU. No Python loops!

def convert_fake_to_data_vectorized(x_fake_tensor, adj_fake_logits, t_embed_batch, device, gumbel=False, temperature=0.5):
    """
    Converts Generator output to a single batched PyG Data object *on the GPU*.
    This is the core optimization, replacing the slow Python loops.
    """
    batch_size, num_nodes, _ = x_fake_tensor.shape
    
    # 1. Create Batched Node Features (x)
    # Reshape [B, N, F] -> [B*N, F]
    x_batched = x_fake_tensor.reshape(batch_size * num_nodes, -1)
    
    # 2. Create Batched Batch Index (batch)
    # Create [0, 0, ..., 1, 1, ..., B-1, B-1]
    batch_vec = torch.arange(batch_size, device=device).repeat_interleave(num_nodes)
    
    # 3. Create Batched Edge Index (edge_index)
    # Repeat the [2, N*(N-1)] template B times
    edge_index_batched = EDGE_INDEX_TEMPLATE.repeat(1, batch_size)
    # Create offsets: [0, 0, ..., N, N, ..., 2N, 2N, ...]
    offset = torch.arange(0, batch_size * num_nodes, num_nodes, device=device).repeat_interleave(NUM_DENSE_EDGES)
    # Add offsets to create the full [2, B*N*(N-1)] edge_index
    edge_index_batched = edge_index_batched + offset
    
    # 4. Create Batched Edge Attributes (edge_attr)
    # We need to gather the logits corresponding to our new edge_index
    # Get batch, row, and column indices from the batched edge_index
    batch_indices = edge_index_batched[0] // num_nodes
    row_indices = edge_index_batched[0] % num_nodes
    col_indices = edge_index_batched[1] % num_nodes
    
    # Gather the [B*N*(N-1), Bonds] logits
    adj_logits_flat = adj_fake_logits[batch_indices, row_indices, col_indices]
    
    # 5. Sample edge attributes
    if gumbel:
        # Differentiable Gumbel-Softmax for Generator step
        edge_attr_batched = F.gumbel_softmax(adj_logits_flat, tau=temperature, hard=True)
    else:
        # Non-differentiable argmax for Discriminator step
        bond_indices = torch.argmax(adj_logits_flat, dim=-1)
        edge_attr_batched = F.one_hot(bond_indices, num_classes=BOND_FEAT_DIM).float()
        
    # 6. Create the single, batched Data object
    fake_data = Data(
        x=x_batched,
        edge_index=edge_index_batched,
        edge_attr=edge_attr_batched,
        batch=batch_vec,
        target_embed=t_embed_batch  # Already [B, T_EMBED_DIM]
    )
    return fake_data


# --- 5.3. WGAN-GP Gradient Penalty (No changes) ---
def calculate_gradient_penalty(discriminator, real_data, fake_data, lambda_gp, device):
    """Calculates the Gradient Penalty on interpolated node features (X)."""
    real_x = real_data.x.detach()
    fake_x = fake_data.x.detach()
    real_x_size = real_x.size(0)
    
    if fake_x.size(0) > real_x_size:
        fake_x = fake_x[:real_x_size]
    elif fake_x.size(0) < real_x_size:
        padding = torch.zeros(real_x_size - fake_x.size(0), fake_x.size(1), device=device)
        fake_x = torch.cat([fake_x, padding], dim=0)

    alpha = torch.rand(real_x_size, 1).to(device) 
    interpolated_x = (alpha * real_x) + ((1 - alpha) * fake_x)
    interpolated_x.requires_grad_(True)

    interpolated_data = Data(x=interpolated_x, 
                             edge_index=real_data.edge_index, 
                             edge_attr=real_data.edge_attr, 
                             batch=real_data.batch, 
                             target_embed=real_data.target_embed)

    disc_interpolates = discriminator(interpolated_data)
    
    gradients = torch.autograd.grad(
        outputs=disc_interpolates, inputs=interpolated_x,
        grad_outputs=torch.ones_like(disc_interpolates),
        create_graph=True, retain_graph=True
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_norm = gradients.norm(2, dim=1)
    gradient_penalty = lambda_gp * ((gradient_norm - 1) ** 2).mean()
    
    return gradient_penalty

# --- 6. Main Training Loop (FIXED & OPTIMIZED) ---
def run_wgan_gp_training(generator, discriminator, data_loader, epochs, n_critic):
    
    generator.train()
    discriminator.train()
    
    for epoch in range(1, epochs + 1):
        g_loss_sum, d_loss_sum = 0, 0
        
        # Use a for loop that automatically prints progress
        from tqdm import tqdm
        progress_bar = tqdm(data_loader, desc=f"Epoch {epoch}/{EPOCHS}")
        
        for batch_idx, real_data in enumerate(progress_bar):
            # OPTIMIZATION: Move real_data to GPU here
            real_data = real_data.to(DEVICE)
            batch_size = real_data.num_graphs
            
            # target_embed is already [B, T_EMBED_DIM] from the loader
            target_embed_batch = real_data.target_embed

            # 1. Train Discriminator (n_critic steps)
            for _ in range(n_critic):
                optimizer_D.zero_grad()
                
                # Real Loss
                d_real = discriminator(real_data).mean()
                
                # Fake Loss
                z = torch.randn(batch_size, Z_DIM).to(DEVICE)
                x_fake, adj_fake_logits = generator(z, target_embed_batch)
                
                # --- D-step: Use VECTORIZED function (non-differentiable) ---
                with torch.no_grad(): # Ensure no grads are computed here
                    fake_data = convert_fake_to_data_vectorized(
                        x_fake.detach(), adj_fake_logits.detach(), target_embed_batch, DEVICE, gumbel=False
                    )
                
                d_fake = discriminator(fake_data).mean()
                
                # Gradient Penalty
                gp = calculate_gradient_penalty(discriminator, real_data, fake_data, LAMBDA_GP, DEVICE)
                
                d_loss = - (d_real - d_fake) + gp
                d_loss.backward()
                optimizer_D.step()
                d_loss_sum += d_loss.item()
            
            # 2. Train Generator (1 step)
            optimizer_G.zero_grad()
            
            z = torch.randn(batch_size, Z_DIM).to(DEVICE)
            x_fake, adj_fake_logits = generator(z, target_embed_batch)
            
            # --- G-step: Use VECTORIZED function (DIFFERENTIABLE) ---
            fake_data = convert_fake_to_data_vectorized(
                x_fake, adj_fake_logits, target_embed_batch, DEVICE, gumbel=True
            )
            
            # Generator Loss
            g_loss = - discriminator(fake_data).mean()
            g_loss.backward()
            optimizer_G.step()
            g_loss_sum += g_loss.item()
            
            # Update progress bar
            progress_bar.set_postfix(
                D_Loss=f"{(d_loss_sum / (batch_idx+1) / n_critic):.4f}", 
                G_Loss=f"{(g_loss_sum / (batch_idx+1)):.4f}"
            )
            
        avg_d_loss = d_loss_sum / len(data_loader) / n_critic
        avg_g_loss = g_loss_sum / len(data_loader)
        # TQDM handles the epoch printout
        # print(f"Epoch {epoch}/{EPOCHS} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")

# --- Execute Training ---
print("\n--- Starting WGAN-GP Training ---")
# Add 'tqdm' to your environment: pip install tqdm
run_wgan_gp_training(generator, discriminator, real_loader, EPOCHS, N_CRITIC) 
print("\nTraining completed.")

✅ CUDA is available! GPU will be used for training.
PyTorch CUDA Version: 12.1
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU
Using device: cuda
Opening gzipped FASTA file: DL_ENDSEM__DATASET/chembl_35_blast.fa.gz
Loading ProtT5 model... (This may take a moment)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Generated protein embedding of shape: torch.Size([1024])
Found 3989 potent inhibitors for UniProt ID P00533.


c:\Users\nikhi\OneDrive\Attachments\Desktop\Git_Repos\NNDL_endsem\venv\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Prepared 1334 real graph samples for training.
Initializing models...

--- Starting WGAN-GP Training ---


Epoch 1/10:   0%|          | 0/21 [00:00<?, ?it/s]c:\Users\nikhi\OneDrive\Attachments\Desktop\Git_Repos\NNDL_endsem\venv\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ..\aten\src\ATen\cuda\CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Epoch 10/10: 100%|██████████| 21/21 [00:20<00:00,  1.04it/s, D_Loss=-327.0524, G_Loss=-381.8533]


Training completed.


In [3]:
# ============================================================
# 📊 DATASET MONITORING
# ============================================================
print("\n" + "="*60)
print("📊 DATASET SUMMARY")
print("="*60)
print(f"✅ Valid molecules: {len(real_data_list)}")
print(f"📦 Batch size: {BATCH_SIZE}")
print(f"🔄 Batches per epoch: {len(real_loader)}")
print(f"🧬 Atoms per molecule: {sum(d.x.size(0) for d in real_data_list)/len(real_data_list):.1f} (avg)")
print(f"🔗 Bonds per molecule: {sum(d.edge_index.size(1) for d in real_data_list)/len(real_data_list):.1f} (avg)")
print(f"⚙️  Target protein: {TARGET_UNIPROT_ID}")
print(f"🎯 Potency cutoff: ≤ 100 nM")
print("="*60 + "\n")
# ============================================================


📊 DATASET SUMMARY
✅ Valid molecules: 1334
📦 Batch size: 64
🔄 Batches per epoch: 21
🧬 Atoms per molecule: 25.7 (avg)
🔗 Bonds per molecule: 56.4 (avg)
⚙️  Target protein: P00533
🎯 Potency cutoff: ≤ 100 nM

